
![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/public/NER_HE.ipynb)

# **Recognize entities in Hebrew text**

# **Colab Setup**

In [ ]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.3.0 spark-nlp==4.2.8

# Install Spark NLP Display lib
! pip install --upgrade -q spark-nlp-display

# **Start the Spark session**

In [2]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *

import pandas as pd
from pyspark.sql.types import StringType, IntegerType

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

import string
import numpy as np

In [3]:
# Start Spark session
spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

Spark NLP version 4.2.8
Apache Spark version: 3.3.0


# **🔎 For about model**


📌 **"hebrewner_cc_300d"**--> *This model uses Hebrew word embeddings to find 10 different types of entities in Hebrew text. It is trained using hebrewner_cc_300d word embeddings, please use the same embeddings in the pipeline.*

*  Persons-PERS, Dates-DATE, Organizations-ORG, Locations-LOC, Percentage-PERCENT, Money-MONEY, Time-TIME, Miscellaneous (Affiliation)-MISC_AFF, Miscellaneous (Event)-MISC_EVENT, Miscellaneous (Entity)-MISC_ENT.





# **🔎Define Spark NLP pipeline**

In [4]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

embeddings = WordEmbeddingsModel.pretrained("hebrew_cc_300d", "he") \
    .setInputCols(["document", "token"]) \
    .setOutputCol("embeddings")

ner = NerDLModel.pretrained("hebrewner_cc_300d", "he" ) \
    .setInputCols(["document", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = NerConverter() \
    .setInputCols(["document", "token", "ner"]) \
    .setOutputCol("ner_chunk")

nlpPipeline = Pipeline(
    stages = [
        documentAssembler, 
        tokenizer,
        embeddings,
        ner,
        ner_converter])

empty_df = spark.createDataFrame([['']]).toDF("text")
pipelineModel = nlpPipeline.fit(empty_df)
light_model = LightPipeline(pipelineModel)

hebrew_cc_300d download started this may take some time.
Approximate size to download 1.1 GB
[OK!]
hebrewner_cc_300d download started this may take some time.
Approximate size to download 14.1 MB
[OK!]


# **🔎Sample text**

In [5]:
text = """והתוצאה : ספרו הפך לרב מכר ענק ובסיס לוויכוחים תיאולוגיים ודיונים נזעמים , כמו גם התקפות והאשמות כלפי בראון מחוגי הכנסייה כפי שמעולם לא התעוררו כתוצאה מספריהם של וואלאס או לאדלום , ואף גרם לסופר מצליח בזכות עצמו , דן בורסטין , לערוך את הספר " הסודות שמאחורי צופן דה וינצ'י " , שבו הוא בודק אחת לאחת את העובדות וההנחות שעליהן מסתמך בראון על ידי שפע של מאמרים , חלקם מקוריים וחלקם לקוחים מספרים , כתבי עת וראיונות עם חוקרים שונים ."""

# **🔎Run the pipeline**

In [6]:
light_result = light_model.fullAnnotate(text)

chunks = []
entities = []
begin =[]
end = []

for n in light_result[0]['ner_chunk']:
        
    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    
import pandas as pd

result_df = pd.DataFrame({'chunks':chunks, 'entities':entities,
                                   'begin': begin, 'end': end})
result_df

Before _validateStagesInputCols


,chunks,entities,begin,end
0,בראון,PERS,102,106
1,וואלאס,PERS,162,167
2,לאדלום,PERS,172,177
3,דן בורסטין,PERS,214,223
4,הסודות שמאחורי צופן דה וינצ'י,MISC_ENT,243,271
5,בראון,PERS,331,335


# **🔎Visualize results**

In [7]:
from sparknlp_display import NerVisualizer

NerVisualizer().display(
    result = light_result[0],
    label_col = 'ner_chunk',
    document_col = 'document'
)